# IsoelectricPoint.jl
---
### Author : Manas Mahale <<manas.mahale@bcp.edu.in>>

Calculate isoelectric points of polypeptides using methods of Bjellqvist. \
pK values and the methods are taken from 
- Bjellqvist, B.,Hughes, G.J., Pasquali, Ch., Paquet, N., Ravier, F.,\
    Sanchez, J.-Ch., Frutiger, S. & Hochstrasser, D.F.\
    The focusing positions of polypeptides in immobilized pH gradients can be\
    predicted from their amino acid sequences. Electrophoresis 1993, 14,\
    1023-1031.
- Bjellqvist, B., Basse, B., Olsen, E. and Celis, J.E.\
    Reference points for comparisons of two-dimensional maps of proteins from\
    different human cell types defined in a pH scale where isoelectric points\
    correlate with polypeptide compositions. Electrophoresis 1994, 15, 529-539.\

The algorithm is designed according to a note by David L. Tabb, available at:\
http://fields.scripps.edu/DTASelect/20010710-pI-Algorithm.pdf \
\
This is an adaptation of BioPython's [IsoelectricPoint.py](https://github.com/biopython/biopython/blob/master/Bio/SeqUtils/IsoelectricPoint.py)
---

In [10]:
using Test

---
# Constants
---

In [2]:
positive_pKs = Dict("Nterm" => 7.5, "K" => 10.0, "R" => 12.0, "H" => 5.98)

negative_pKs = Dict("Cterm" => 3.55, "D" => 4.05, "E" => 4.45, "C" => 9.0, "Y" => 10.0)

pKcterminal = Dict("D" => 4.55, "E" => 4.75)

pKnterminal = Dict(
    "A" => 7.59,
    "M" => 7.0,
    "S" => 6.93,
    "P" => 8.36,
    "T" => 6.82,
    "V" => 7.44,
    "E" => 7.7)

charged_aas = ("K", "R", "H", "D", "E", "C", "Y")

protein_letters = "ACDEFGHIKLMNPQRSTVWY"

"ACDEFGHIKLMNPQRSTVWY"

---
# Utils
---

In [3]:
function count_amino_acids(sequence)
    c = Dict()
    for i in protein_letters
        c[string(i)] = count(string(i), sequence)
    end
    return c
end


function _select_charged(aa_content)
    charged = Dict()
  
    for aa in charged_aas
        charged[aa] = convert(Float64, aa_content[aa])
    end

    charged["Nterm"] = 1.0
    charged["Cterm"] = 1.0
    return charged
end


function _update_pKs_tables(sequence)
    pos_pKs = deepcopy(positive_pKs)
    neg_pKs = deepcopy(negative_pKs)
    nterm, cterm = sequence[1], sequence[end]
    if nterm in keys(pKnterminal)
        pos_pKs["Nterm"] = pKnterminal[nterm]
    end
    if cterm in keys(pKcterminal)
        neg_pKs["Cterm"] = pKcterminal[cterm]
    end
    return pos_pKs, neg_pKs
end

_update_pKs_tables (generic function with 1 method)

---
# Charge at given pH
---

In [4]:
function charge_at_pH(pH)
    positive_charge = 0.0
    for (aa, pK) in pos_pKs
        partial_charge = 1.0 / (10^(pH - pK) + 1.0)
        positive_charge += charged_aas_content[aa] * partial_charge
    end
    negative_charge = 0.0
    for (aa, pK) in neg_pKs
        partial_charge = 1.0 / (10^(pK - pH) + 1.0)
        negative_charge += charged_aas_content[aa] * partial_charge
    end
    return positive_charge - negative_charge
end

charge_at_pH (generic function with 1 method)

---
# Calculate Isoelectric Point
---


In [5]:
function pI(pH=7.775, min_=4.05, max_=12)
    charge = charge_at_pH(pH)
    if max_ - min_ > 0.0001
        if charge > 0.0
            min_ = pH
        else
            max_ = pH
        end
        next_pH = (min_ + max_) / 2
        return pI(next_pH, min_, max_)
    end
    return pH
end

pI (generic function with 4 methods)

---
# Tests
---

In [6]:
protein_sequence = "PETER"
sequence = uppercase(protein_sequence)
aa_content = count_amino_acids(sequence)
charged_aas_content = _select_charged(aa_content)
pos_pKs, neg_pKs = _update_pKs_tables(sequence)
@test pI() == 4.531397819519043

Test Passed

In [7]:
@test charge_at_pH(4.531397819519043) == -1.5946248781428807e-6

Test Passed

In [8]:
protein_sequence = "INGAR"
sequence = uppercase(protein_sequence)
aa_content = count_amino_acids(sequence)
charged_aas_content = _select_charged(aa_content)
pos_pKs, neg_pKs = _update_pKs_tables(sequence)
@test pI() == 9.750021171569824

Test Passed

In [9]:
@test charge_at_pH(7) == 0.7600916142893016

Test Passed